In [53]:
from bayes_opt import BayesianOptimization
from xgboost import XGBRegressor
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

###############################################
# Import Miscellaneous Assets
###############################################
# Data manipulation
import pandas as pd
import numpy as np

# Modeling
import lightgbm as lgb

# Evaluation of the model
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pandas.plotting import scatter_matrix
%matplotlib inline


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from dateutil.parser import parse
from datetime import datetime
from scipy.stats import norm

# import all what you need for machine learning
import sklearn
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
import warnings
from datetime import datetime
from functools import partial
from pprint import pprint as pp
from tqdm import tqdm, tqdm_notebook

import tensorflow as tf
import math
from hyperopt import hp

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

import xgboost as xgb
from sklearn.metrics import accuracy_score

from numpy import loadtxt
from xgboost import XGBClassifier

import lightgbm as lgb
from hyperopt import STATUS_OK

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers

from sklearn.metrics import accuracy_score
import os
from sklearn import preprocessing

import h2o
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators import H2OXGBoostEstimator
###### ESSIENTIAL CODE ###########
train = pd.read_csv("/kaggle/input/datamaestro2020/astro_train.csv")
test = pd.read_csv("/kaggle/input/datamaestro2020/astro_test.csv")
sample = pd.read_csv("/kaggle/input/datamaestro2020/sample_submission.csv")

### useless thing ifigured all out in different notebook, data_maestro_1 dekho ab
del train["id"]
del test["id"]

del train["rerun"]
del test["rerun"]
del train["skyVersion"]
del test["skyVersion"]
del train["run"]
del test["run"]
del train["camCol"]
del test["camCol"]

### minmax scaling karna hai but it gets converted to a different data type 
### bhavika tera manually kiya hua function bhej lol

train.to_csv('train_new.csv', header=True, index=False)
test.to_csv('test_new.csv', header=True, index=False)

In [54]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from bayes_opt import BayesianOptimization
h2o.init()
h2o.remove_all()

data = h2o.upload_file("/kaggle/working/train_new.csv")
train_cols = [x for x in data.col_names if x not in ['class']]
target = "class"
train, test = data.split_frame(ratios=[0.7])

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 hour 3 mins
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.4
H2O cluster version age:,19 days
H2O cluster name:,H2O_from_python_unknownUser_uwpswn
H2O cluster total nodes:,1
H2O cluster free memory:,2.641 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [64]:
def train_model(learn_rate, 
                max_depth,
                min_child_weight, 
                gamma,
                colsample_bytree,
                eta):
    params = {
        'learn_rate': learn_rate ,
        'max_depth' :int(max_depth) ,
        'min_child_weight' :int(min_child_weight) ,
        'gamma' : gamma,
        'colsample_bytree' : colsample_bytree,
        'eta':eta
    }
    model = H2OXGBoostEstimator(nfolds=5,**params)
    model.train(x=train_cols, y=target, training_frame=train)
    return -model.rmse()

In [65]:
bounds = {
        'learn_rate': (0.01,0.3) ,
        'max_depth' :(3,15) ,
        'min_child_weight' :(1,7) ,
        'gamma' : (0,0.4),
        'colsample_bytree' : (0.3,1.0),
        'eta':(0.01,0.2)
    }

In [66]:
optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=1,
)
optimizer.maximize(init_points=10, n_iter=100)
# n_iter = 50 #basic

|   iter    |  target   | colsam... |    eta    |   gamma   | learn_... | max_depth | min_ch... |
-------------------------------------------------------------------------------------------------
xgboost Model Build progress: |███████████████████████████████████████████| 100%
|  1        | -0.664    |  0.5919   |  0.1469   |  4.575e-0 |  0.09768  |  4.761    |  1.554    |
xgboost Model Build progress: |███████████████████████████████████████████| 100%
|  2        | -0.5978   |  0.4304   |  0.07566  |  0.1587   |  0.1663   |  8.03     |  5.111    |
xgboost Model Build progress: |███████████████████████████████████████████| 100%
|  3        | -0.5282   |  0.4431   |  0.1768   |  0.01096  |  0.2044   |  8.008    |  4.352    |
xgboost Model Build progress: |███████████████████████████████████████████| 100%
|  4        | -0.6733   |  0.3983   |  0.04764  |  0.3203   |  0.2908   |  6.761    |  5.154    |
xgboost Model Build progress: |███████████████████████████████████████████| 100%
|  5   

H2OJobCancelled: Job<$03017f00000132d4ffffffff$_aae870d11ee66b97970787b12f5c442b> was cancelled by the user.

In [67]:
optimizer.max

{'target': -0.2016664166253024,
 'params': {'colsample_bytree': 0.7805506493771085,
  'eta': 0.16857887766050086,
  'gamma': 0.007315310937676723,
  'learn_rate': 0.22754185133404056,
  'max_depth': 14.866333066877937,
  'min_child_weight': 5.488993926279036}}

In [68]:
#set manually to optimiser.max idk how to do directly cheap trick lol
#maxdepth and ntrees int karaychi approx

#defining
model = H2OXGBoostEstimator(colsample_bytree=0.7805506493771085, 
                                     eta = 0.16857887766050086,
                                     gamma  = 0.007315310937676723,
                                     learn_rate = 0.22754185133404056,
                                     max_depth = 15,
                                     min_child_weight = 5
                                    )

#traneeeee
model.train(x=train_cols, y=target, training_frame=train)

xgboost Model Build progress: |███████████████████████████████████████████| 100%


In [69]:
test = h2o.upload_file("/kaggle/working/test_new.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [71]:
#a lotta random convesions here which i tuk too long to figar out

predictions = model.predict(test)
pdr = predictions.as_data_frame().to_numpy()
p = np.round(pdr).astype(int)

#allah ho gaya finally

submission_df = pd.DataFrame(columns=['id', 'class'])
submission_df['id'] = sample['id']
submission_df['class'] = p
submission_df.to_csv('bayesian.csv', header=True, index=False)

#dekhlo ab kyahi, test noi banaya banana hai toh use own brain
submission_df.head(100)

xgboost prediction progress: |████████████████████████████████████████████| 100%


,id,class
0,45000,1
1,45001,0
2,45002,1
3,45003,0
4,45004,1
...,...,...
95,45095,1
96,45096,0
97,45097,1
98,45098,1
